## MBA em Ciência de Dados
# Redes Neurais e Arquiteturas Profundas

### <span style="color:darkred">Módulo 7 - Auto-encodes e Redes Geradoras
</span>

#### <span style="color:darkred">**Parte 2: Autoencoders para Redução de Dimensionalidade**</span>

Moacir Antonelli Ponti

CeMEAI - ICMC/USP São Carlos

---

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from numpy.random import seed
from tensorflow.random import set_seed
from tensorflow import keras
from tensorflow.keras import layers

Outra aplicação de auto-encoders é seu uso para **redução de dimensionalidade não supervisionada**

Vamos utilizar a base de dados Boston Housing e a tarefa de regressão

Essa base de dados possui 13 atributos originalmente, vamos aprender uma redução desse espaço.

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.boston_housing.load_data()

mean = x_train.mean(axis=0)
x_train -= mean
std = x_train.std(axis=0)
x_train /= std

x_test -= mean
x_test /= std
print(x_test.shape)

target_dimensions = 6

57026/57026 [==============================] - 0s 0us/step
(102, 13)


### O autoencoder será denso, com:

* uma camada de 10 dimensões intermediária (para o encoder e o decoder)
* a camada de projeção no espaco do código

In [ ]:
input_data = keras.layers.Input(shape=(13,))

# encoder
e1 = keras.layers.Dense(10, activation='tanh')(input_data)
z = keras.layers.Dense(target_dimensions, activation='tanh',
                       name='code')(e1)

# decoder
d1 = keras.layers.Dense(10, activation='tanh')(z)
output = keras.layers.Dense(13, activation='tanh')(d1)

autoencoder = keras.models.Model(input_data, output)
autoencoder.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 13)]              0         
                                                                 
 dense_7 (Dense)             (None, 10)                140       
                                                                 
 code (Dense)                (None, 6)                 66        
                                                                 
 dense_8 (Dense)             (None, 10)                70        
                                                                 
 dense_9 (Dense)             (None, 13)                143       
                                                                 
Total params: 419 (1.64 KB)
Trainable params: 419 (1.64 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
seed(1)
set_seed(1)

epochs = 150
batch_size = 8

# definindo um decaimento para a taxa de aprendizado
def scheduler(epoch, lr):
  if epoch < 50:
    return lr
  else:
    return float(lr * tf.math.exp(-0.01))

callbacklr = tf.keras.callbacks.LearningRateScheduler(scheduler)

autoencoder.compile(loss='mse',
              optimizer=keras.optimizers.Adam(learning_rate=0.003))

hist_ae = autoencoder.fit(x_train, x_train,
                    callbacks=[callbacklr],
                    batch_size=batch_size, epochs=epochs,
                    verbose=2)

Epoch 1/150
51/51 - 1s - loss: 0.8801 - lr: 0.0030 - 868ms/epoch - 17ms/step
Epoch 2/150
51/51 - 0s - loss: 0.5754 - lr: 0.0030 - 94ms/epoch - 2ms/step
Epoch 3/150
51/51 - 0s - loss: 0.4932 - lr: 0.0030 - 96ms/epoch - 2ms/step
Epoch 4/150
51/51 - 0s - loss: 0.4448 - lr: 0.0030 - 91ms/epoch - 2ms/step
Epoch 5/150
51/51 - 0s - loss: 0.4129 - lr: 0.0030 - 88ms/epoch - 2ms/step
Epoch 6/150
51/51 - 0s - loss: 0.3925 - lr: 0.0030 - 95ms/epoch - 2ms/step
Epoch 7/150
51/51 - 0s - loss: 0.3776 - lr: 0.0030 - 89ms/epoch - 2ms/step
Epoch 8/150
51/51 - 0s - loss: 0.3654 - lr: 0.0030 - 90ms/epoch - 2ms/step
Epoch 9/150
51/51 - 0s - loss: 0.3553 - lr: 0.0030 - 104ms/epoch - 2ms/step
Epoch 10/150
51/51 - 0s - loss: 0.3463 - lr: 0.0030 - 90ms/epoch - 2ms/step
Epoch 11/150
51/51 - 0s - loss: 0.3377 - lr: 0.0030 - 90ms/epoch - 2ms/step
Epoch 12/150
51/51 - 0s - loss: 0.3311 - lr: 0.0030 - 90ms/epoch - 2ms/step
Epoch 13/150
51/51 - 0s - loss: 0.3252 - lr: 0.0030 - 84ms/epoch - 2ms/step
Epoch 14/150
51/51

---

### Uso em regressor externo

Com o AE treinado, podemos utilizá-lo para obter representações para instâncias do treinamento e do teste

Vamos compara esse novo espaço aprendido com o original e com uma projeção PCA

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.decomposition import PCA
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:
code_model = keras.models.Model(inputs=autoencoder.input,
                                outputs=autoencoder.get_layer('code').output)
code_train = np.asarray(code_model.predict(x_train))
code_test  = np.asarray(code_model.predict(x_test))
print("Training data size = ", code_train.shape)
print("Testing data size = ", code_test.shape)

4/4 [==============================] - 0s 3ms/step
Training data size =  (404, 6)
Testing data size =  (102, 6)


Criando a projeção PCA para comparação

In [ ]:
pca = PCA(n_components=target_dimensions)
pca.fit(x_train)
pca_train = pca.transform(x_train)
pca_test = pca.transform(x_test)

Treinando os regressores

In [ ]:
print('Treinando Regressor com Código AE...')
clf_ae = Ridge()
clf_ae.fit(code_train, y_train)
code_pred = clf_ae.predict(code_test)

Treinando Regressor com Código AE...


In [ ]:
print('Treinando Regressor com PCA...')
clf_pca = Ridge()
clf_pca.fit(pca_train, y_train)
pca_pred = clf_pca.predict(pca_test)

Treinando Regressor com PCA...


In [ ]:
print('Treinando Regressor com Dados Originais...')
clf_ori = Ridge()
clf_ori.fit(x_train, y_train)
y_pred = clf_ori.predict(x_test)

Treinando Regressor com Dados Originais...


In [ ]:
print('Calculando score...')
score_ae = mean_squared_error(code_pred, y_test)
score_ori = mean_squared_error(y_pred, y_test)
score_pca = mean_squared_error(pca_pred, y_test)
print('\nscore original: %.2f' % (score_ori))
print('\nscore PCA: %.2f' % (score_pca))
print('\nscore AE: %.2f ' % (score_ae))


Calculando score...

score original: 23.11

score PCA: 21.63

score AE: 20.67 
